<div style="text-align: right" align="right"><i>Peter Norvig<br>December 2020</i></div>

# Advent of Code 2020

This year I return to [Advent of Code](https://adventofcode.com), as I did in [2016](Advent+of+Code), [17](Advent+2017), and [18](Advent-2018.ipynb). Thank you, [Eric Wastl](http://was.tl/)! This notebook describes each day's puzzle only briefly; you'll have to look at the [Advent of Code website](https://adventofcode.com/2020) if you want the full details. Each puzzle has a part 1 and a part 2.

For each day from 1 to 25, I'll write **four pieces of code** with the following format (and perhaps some auxiliary code). For example, on day 3:
- `in3: List[str] = data(3)`: the day's input data, parsed into an appropriate form (here, a list of string lines). Most days the data comes from a file, read via the function `data`, but some days the data is so small I just copy and paste it.
- `def day3_1(nums): ...   `: a function that takes the day's data as input and returns the answer for part 1.
- `def day3_2(nums): ...   `: a function that takes the day's data as input and returns the answer for part 2.
- `do(3)`: runs `day3_1(in3)`. I'll then use the result to hopefully unlock part 2 and define `day3_2`, which also gets run when I call `do(3)` again. Once I verify both answers, I'll change `do(3)` to `do(3, 167, 736527114)` to serve as a unit test.

# Day 0: Imports and Utility Functions

Preparations prior to Day 1:
- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.

Since I'm not even attempting to compete for speed, I'll take the time to use reasonable variable names (not single-letter names), and to give type annotations for most of the functions I define (but not the `day` functions, which all return `int`). Traditionally, a lot of problems are of the form "How many of your input items have property P?", for which I can do `quantify(inputs, P)` or "What is the sum of the result of applying F to each input item?", for which I can do `sum(map(F, inputs))`.



In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

In [2]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/advent2020/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> Dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = {}
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got[part] = g[fname](g[f'in{day}'])
            if len(answers) >= part: 
                assert got[part] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[part]}; expected {answers[part - 1]}')
    return got

In [3]:
def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def multimap(items: Iterable[Tuple]) -> dict:
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def prod(numbers) -> float: # Will be math.prod in Python 3.8, but I'm in 3.7
    "The product of an iterable of numbers." 
    result = 1
    for n in numbers:
        result *= n
    return result

def ints(text: str) -> Tuple[int]:
    "Return a tuple of all the integers in text."
    return tuple(map(int, re.findall('-?[0-9]+', text)))

cat = ''.join
flatten = chain.from_iterable
Char = str # Type used to indicate a single character

# Day 1: Report Repair

1. Find the two entries in your expense report (a file of integers) that sum to 2020; what do you get if you multiply them together?
2.  In your expense report, what is the product of the three entries that sum to 2020?

In [4]:
in1: Set[int] = set(data(1, int))

In [5]:
def day1_1(nums):
    "Find 2 distinct numbers that sum to 2020, and return their product."
    return first(x * y 
                 for x in nums 
                 for y in nums & {2020 - x} 
                 if x != y)

In [6]:
def day1_2(nums):
    "Find 3 distinct numbers that sum to 2020, and return their product."
    return first(x * y * z 
                 for x, y in combinations(nums, 2) 
                 for z in nums & {2020 - x - y} 
                 if x != z != y)

In [7]:
do(1, 787776, 262738554)

{1: 787776, 2: 262738554}

# Day 2: Password Philosophy

1. A password policy is of the form "`1-3 b: cdefg`" meaning that the password must contain 1 to 3 instances of `b`; `cdefg` is invalid under this policy. How many passwords in your input file are valid according to their policies?
-  JK! The policy actually means that exactly one of positions 1 and 3 (1-based) must contain the letter `b`. How many passwords are valid according to the new interpretation of the policies?

In [8]:
Policy = Tuple[int, int, Char, str]

def parse_password_policy(line: str) -> Policy:
    "Given '1-3 b: cdefg', return (1, 3, 'b', 'cdefg')."
    a, b, L, pw = re.findall(r'[^-:\s]+', line)
    return (int(a), int(b), L, pw)

in2: List[tuple] = data(2, parse_password_policy)

In [9]:
def valid_password(policy) -> bool: 
    "Does policy's pw have between a and b instances of letter L?"
    a, b, L, pw = policy
    return a <= pw.count(L) <= b

def day2_1(policies): return quantify(policies, valid_password)

In [10]:
def day2_2(policies): return quantify(policies, valid_password_2)

def valid_password_2(policy) -> bool: 
    "Does line's pw have letter L at position a or b (1-based), but not both?"
    a, b, L, pw = policy
    return (L == pw[a - 1]) ^ (L == pw[b - 1])

In [11]:
do(2, 383, 272)

{1: 383, 2: 272}

# Day 3: Toboggan Trajectory

The input file is a map of a field that looks like this:

    ..##.......
    #...#...#..
    .#....#..#.
    ..#.#...#.#
    .#...##..#.
    
where each `#` is a tree and the pattern in each row implicitly repeats to the right.

1. Starting at the top-left corner of your map and following a slope of down 1 and right 3, how many trees would you encounter?
2. What do you get if you multiply together the number of trees encountered on each of the slopes 1/1, 1/3, 1/5, 1/7, 2/1?


In [12]:
Picture = List[str]

in3: Picture = data(3)

In [13]:
def day3_1(rows, dx=3, dy=1, tree='#'): 
    "How many trees are on the coordinates on the slope dy/dx?"
    return quantify(row[(dx * y) % len(row)] == tree
                    for y, row in enumerate(rows[::dy]))

In [14]:
def day3_2(rows):
    "What is the product of the number of trees on these five slopes?"
    def t(dx, dy): return day3_1(rows, dx, dy)
    return t(1, 1) * t(3, 1) * t(5, 1) * t(7, 1) * t(1, 2) 

In [15]:
do(3, 167, 736527114)

{1: 167, 2: 736527114}

# Day 4: Passport Processing

The input is a file of passport data that looks like this (each passport is a series of field:value pairs separated from the next passport by a blank line):

    ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
    byr:1937 iyr:2017 cid:147 hgt:183cm

    iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884 hcl:#cfa07d byr:1929

    hcl:#ae17e1 iyr:2013
    eyr:2024 ecl:brn pid:760753108 byr:1931 hgt:179cm
    
    
1. Count the number of valid passports &mdash; those that have all seven required fields (`byr, ecl, eyr, hcl, hgt, iyr, pid`). 
2. Count the number of valid passports &mdash; those that have valid values for all required fields (see the rules in `valid_fields`). 

In [16]:
Passport = dict # e.g. {'iyr': '2013', ...}

def parse_passport(text: str) -> Passport:
    "Make a dict of the 'key:val' entries in text."
    return Passport(re.findall(r'([a-z]+):([^\s]+)', text))

assert parse_passport('''a:1 b:two\nsee:3''') == {'a': '1', 'b': 'two', 'see': '3'}

in4: List[Passport] = data(4, parse_passport, '\n\n') # Passports are separated by blank lines

In [17]:
required_fields = {'byr', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}

def valid_passport(passport) -> bool: return required_fields.issubset(passport)

def day4_1(passports): return quantify(passports, valid_passport)    

In [18]:
def day4_2(passports): return quantify(passports, valid_passport_fields)

def valid_passport_fields(passport) -> bool:
    '''Validate fields according to the following rules:
    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expr. Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
      If cm, the number must be at least 150 and at most 193.
      If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a '#' followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.'''
    return (valid_passport(passport)
            and all(field_validator[field](passport[field])
                    for field in required_fields))

field_validator = dict(
    byr=lambda v: 1920 <= int(v) <= 2002,
    iyr=lambda v: 2010 <= int(v) <= 2020,
    eyr=lambda v: 2020 <= int(v) <= 2030,
    hcl=lambda v: re.match('#[0-9a-f]{6}$', v),
    ecl=lambda v: re.match('(amb|blu|brn|gry|grn|hzl|oth)$', v),
    pid=lambda v: re.match('[0-9]{9}$', v),
    hgt=lambda v: ((v.endswith('cm') and 150 <= int(v[:-2]) <= 193) or
                   (v.endswith('in') and  59 <= int(v[:-2]) <=  76)))

In [19]:
do(4, 237, 172)

{1: 237, 2: 172}

# Day 5: Binary Boarding

The input is a list of boarding passes, such as `BFFFBBFRRR`. Each boarding pass corrsponds to a **seat ID** using an encoding where B and F stand for the back and front half of the remaining part of the plane; R and L stand for right and left half of a row. (The encoding is the same as substituting 0 for F or L, and 1 for B or R, and treating the result as a binary number.)

1. What is the highest seat ID on a boarding pass?
-  What is the one missing seat ID, between the minimum and maximum IDs, that is not on the list of boarding passes?

In [20]:
def seat_id(seat: str, table=str.maketrans('FLBR', '0011')) -> int:
    "Treat a seat description as a binary number; convert to int."
    return int(seat.translate(table), base=2)

assert seat_id('FBFBBFFRLR') == 357

in5: List[int] = data(5, seat_id)

In [21]:
day5_1 = max # Find the maximum seat id.

In [22]:
def day5_2(ids):
    "The one missing seat id."
    [missing] = set(range(min(ids), max(ids))) - set(ids)
    return missing

In [23]:
do(5, 906, 519)

{1: 906, 2: 519}

# Day 6: Custom Customs

Each passenger fills out a customs form; passengers are arranged in groups. The "yes" answer are recorded; each person on one line, each group separated by a blank line. E.g.:

    abc

    a
    b
    c

    ab
    ac
    
1. For each group, count the number of questions to which *anyone* answered "yes". What is the sum of those counts?
2. For each group, count the number of questions to which *everyone* answered "yes". What is the sum of those counts?

In [24]:
in6: List[List[str]] = data(6, str.splitlines, '\n\n')

assert in6[1] == ['arke', 'qzr', 'plmgnr', 'uriq'] # A group is a list of strs

In [25]:
def day6_1(groups): 
    "For each group, compute the number of letters that ANYONE got. Sum them."
    return sum(len(set(cat(group)))
               for group in groups)

In [26]:
def day6_2(groups: List[List[str]]): 
    "For each group, compute the number of letters that EVERYONE got. Sum them."
    return sum(len(set.intersection(*map(set, group)))
               for group in groups)

In [27]:
do(6, 6530, 3323)

{1: 6530, 2: 3323}

# Day 7: Handy Haversacks

There are strict luggage processing rules for what color bags must contain what other bags. For example:

    light red bags contain 1 bright white bag, 2 muted yellow bags.
    dark orange bags contain 3 bright white bags, 4 muted yellow bags.
    bright white bags contain 1 shiny gold bag.
    
1. How many bag colors must eventually contain at least one shiny gold bag?
2. How many individual bags must be inside your single shiny gold bag?

I wasn't quite sure, but it turns out that "light red" and "dark red" are different colors.

In [28]:
Bag = str
BagRules = Dict[Bag, Dict[Bag, int]] # {outer: {inner: count, ...}, ...}

def parse_bag_rule(line: str) -> Tuple[Bag, Dict[Bag, int]]:
    "Return (outer_bag, {inner_bag: num, ...})"
    line = re.sub(' bags?|[.]', '', line) # Remove redundant info
    outer, inner = line.split(' contain ')
    return outer, dict(map(parse_inner, inner.split(', ')))

def parse_inner(text) -> Tuple[Bag, int]:
    "Return the color and number of inner bags."
    n, bag = text.split(maxsplit=1)
    return bag, (0 if n == 'no' else int(n))

assert parse_inner('3 muted gray') == ('muted gray', 3)

assert (dict([parse_bag_rule("shiny gold bags contain 4 bright blue bags")])
        == {'shiny gold': {'bright blue': 4}})

in7: BagRules = dict(data(7, parse_bag_rule))

In [29]:
def day7_1(rules, target='shiny gold'):
    "How many colors of bags can contain the target color bag?"""
    return quantify(contains(bag, target, rules) for bag in rules)

def contains(bag, target, rules) -> bool:
    "Does this bag contain the target (perhaps recursively)?"
    contents = rules.get(bag, {})
    return (target in contents
            or any(contains(inner, target, rules) for inner in contents))

In [30]:
def day7_2(rules, target='shiny gold'): return num_contained_in(target, rules)

def num_contained_in(target, rules) -> int:
    "How many bags are contained (recursively) in the target bag?"
    return sum(n + n * num_contained_in(bag, rules) 
               for (bag, n) in rules[target].items() if n > 0)

In [31]:
do(7, 103, 1469)

{1: 103, 2: 1469}

# Day 8: Handheld Halting

The puzzle input is a program in an assembly language with three instructions: `jmp, acc, nop`. Since there is no conditional branch instruction, a program that executes any instruction twice will infinite loop; terminating programs will execute each instruction at most once.

1. Immediately before any instruction is executed a second time, what value is in the accumulator register?
2. Fix the program so that it terminates normally by changing exactly one jmp to nop or nop to jmp. What is the value of the accumulator register after the program terminates?

In [32]:
Instruction = Tuple[str, int] # e.g. ('jmp', +4)
Program = List[Instruction]

def parse_instruction(line: str) -> Instruction:
    "Parse a line of assembly code into an Instruction: an ('opcode', int) pair."
    opcode, arg = line.split()
    return opcode, int(arg)
    
in8: Program = data(8, parse_instruction)
    
def day8_1(program):
    "Execte the program until it loops; then return accum."
    pc = accum = 0
    executed = set()
    while True:
        if pc in executed:
            return accum
        executed.add(pc)
        opcode, arg = program[pc]
        pc += 1
        if opcode == 'acc':
            accum += arg
        if opcode == 'jmp':
            pc = pc - 1 + arg

I had to think about what to do for Part 2. Do I need to make a flow graph of where the loops are? That sounds hard. But I soon realized that I can just use brute force&mdash;try every alteration of an instruction (there are only $O(n)$ of them), and run each altered program to see if it terminates (that too takes only $O(n)$ time).

In [33]:
def day8_2(program): 
    "Return the accumulator from the first altered program that terminates."
    programs = altered_programs(program)
    return first(accum for (terminates, accum) in map(run_program, programs)
                 if terminates)

def altered_programs(program, other=dict(jmp='nop', nop='jmp')) -> Iterator[Program]:
    "All ways to swap a nop for a jmp or vice-versa."
    for i, (opcode, arg) in enumerate(program):
        if opcode in other:
            yield [*program[:i], (other[opcode], arg), *program[i + 1:]]

def run_program(program) -> Tuple[bool, int]:
    "Run the program until it loops or terminates; return (terminates, accum)"
    pc = accum = 0
    executed = set()
    while 0 <= pc < len(program):
        if pc in executed:
            return False, accum # program loops
        executed.add(pc)
        opcode, arg = program[pc]
        pc += 1
        if opcode == 'acc':
            accum += arg
        if opcode == 'jmp':
            pc = pc - 1 + arg
    return True, accum # program terminates

In [34]:
do(8, 1521, 1016)

{1: 1521, 2: 1016}

# Day 9: Encoding Error

Given a list of numbers:

1. Find the first number in the list (after the preamble of 25 numbers) which is not the sum of two of the 25 numbers before it.
2. Find a contiguous subsequence of numbers in your list which sum to the number from step 1; add the smallest and largest numbers in this subsequence.

I could do this efficiently in $O(n)$ as in Day 1, but $n$ is so small I'll just use brute force.

In [35]:
in9: List[int] = data(9, int)

def day9_1(nums, p=25):
    """Find the first number in the list of numbers (after a preamble of p=25 numbers) 
    which is not the sum of two of the p numbers before it."""
    return first(x for i, x in enumerate(nums) 
                 if i > p and x not in twosums(nums[i-p:i]))

def twosums(nums): return map(sum, combinations(nums, 2))

In [36]:
def day9_2(nums, target=day9_1(in9)):
    "Find a contiguous subsequence of nums that sums to target; add their max and min."
    subseq = find_subseq(nums, target)
    return max(subseq) + min(subseq)

def find_subseq(nums, target) -> Optional[deque]:
    "Find a contiguous subsequence of nums that sums to target."
    subseq = deque()
    total = 0
    for x in nums:
        if total < target:
            subseq.append(x)
            total += x
        if total == target and len(subseq) >= 2:
            return subseq
        while total > target:
            total -= subseq.popleft()
    return None

In [37]:
do(9, 776203571, 104800569)

{1: 776203571, 2: 104800569}

# Day 10: Adapter Array

You are given a bunch of *joltage adapters*, each with a listed joltage output; each can take an input source that is 1, 2, or 3 jolts lower than the output. There is a charging outlet rated 0 jolts, and you want to chargwe a device that is 3 jolts higher than the maximum adapter.

1. Find a chain that uses all of your adapters to connect the charging outlet to your device's built-in adapter and count the joltage differences between the charging outlet, the adapters, and your device. What is the number of 1-jolt differences multiplied by the number of 3-jolt differences?
2. What is the total number of distinct ways you can arrange the adapters to connect the charging outlet to your device?

Note: at first I thought this was a search problem. But then I realized that the only possibility is to increase joltage from source to adapter, so that means the adapters must appear in sorted order. For part 2, some  adapters can be left out.

In [38]:
in10: List[int] = data(10, int) # Input is the joltage of each adapter

def day10_1(jolts):
    """Arrange the joltages in order; count the number of each size difference;
    return the product of 1- and 3-jolt differences."""
    jolts = [0, *sorted(jolts), max(jolts) + 3]
    diffs = Counter(jolts[i + 1] - jolts[i] 
                    for i in range(len(jolts) - 1))
    assert {1, 2, 3}.issuperset(diffs)
    return diffs[1] * diffs[3]

In [39]:
def day10_2(jolts): return arrangements(tuple(sorted(jolts)), 0)

@lru_cache(None)
def arrangements(jolts, prev) -> int:
    """The number of ways to attach some of `jolts` to `prev`."""
    first, rest = jolts[0], jolts[1:]
    if first - prev > 3:
        return 0
    elif not rest:
        return 1
    else:
        return (arrangements(rest, first) + # Use first
                arrangements(rest, prev))   # Skip first
    
assert arrangements((3, 6, 9, 12), 0) == 1
assert arrangements((3, 6, 9, 13), 0) == 0
assert arrangements((1, 2, 3,  4), 0) == 7

In [40]:
do(10, 2346, 6044831973376)

{1: 2346, 2: 6044831973376}

# Day 11: Seating System

This is a version of Conway's *Life*, except that:
- The world is bounded, not infinite. 
- Cells (seats) have three states, not two: *floor* as well as the traditional *occupied* and *empty*.
- The rules for what changes between occupied and empty in the next generation are different: 

   - If a seat is empty (`L`) and there are no occupied seats adjacent to it, the seat becomes occupied.
   - If a seat is occupied (`#`) and four or more seats adjacent to it are also occupied, the seat becomes empty.
   - Otherwise, the seat's state does not change.

   
1. Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?
2. Same problem, but with two rule changes:
  - When considering adjacency, if there is a *floor* cell in some direction, skip over that to the next visible seat in that direction. 
  - Empty a seat only when there are 5 occupied neighbors, not 4. 

In [41]:
in11: List[str] = data(11)

In [42]:
floor, empty, occupied, off = ".L#?"

Contents = Char # The contents of each location is one of the above 4 characters

class Layout(list):
    """A layout of seats (occupied or not) and floor space."""
    
    crowded = 4
    
    deltas = ((-1, -1), (0, -1), (1, -1),
              (-1,  0),          (1,  0),
              (-1, +1), (0, +1), (1, +1))
    
    def next_generation(self) -> Layout:
        "The next generation, according to the rules."
        seats = (cat(self.next_generation_at(x, y) for x in range(len(self[y])))
                 for y in range(len(self)))
        return type(self)(seats)

    def next_generation_at(self, x, y) -> Contents:
        "The contents of location (x, y) in the next generation."
        old = self[y][x]
        N   = self.neighbors(x, y).count(occupied)
        return (occupied if old is empty    and N == 0            else
                empty    if old is occupied and N >= self.crowded else
                old)

    def neighbors(self, x, y) -> List[Contents]: 
        "The contents of the 8 neighboring locations."
        return [self.at(x + dx, y + dy) for dx, dy in self.deltas]

    def count(self, kind: Contents) -> int: return cat(self).count(kind)
    
    def at(self, x, y) -> Contents:
        "The contents of location (x, y): empty, occupied, floor, or off?"
        if 0 <= y < len(self) and 0 <= x < len(self[y]):
            return self[y][x]
        else:
            return off
        
    def run(self) -> Layout:
        "Run until equilibrium."
        new = self
        while True:
            new, old = new.next_generation(), new
            if new == old:
                return new

def day11_1(seats): return Layout(seats).run().count(occupied)

In [43]:
def day11_2(seats): return Layout2(seats).run().count(occupied)

class Layout2(Layout):
    """A layout of seats (occupied or not) and floor space, with new rules."""

    crowded = 5
    
    def neighbors(self, x, y) -> List[Contents]:
        "The contents of the nearest visible seat in each of the 8 directions."
        return [self.visible(x, dx, y, dy) for dx, dy in self.deltas]
    
    def visible(self, x, dx, y, dy) -> Contents:
        "The contents of the first visible seat in direction (dx, dy)."
        for i in range(1, maxsize):
            x += dx; y += dy
            if not (0 <= y < len(self) and 0 <= x < len(self[y])):
                return off
            if self[y][x] is not floor:
                return self[y][x]                

In [44]:
%time do(11, 2299, 2047)

CPU times: user 9.62 s, sys: 30.7 ms, total: 9.65 s
Wall time: 9.7 s


{1: 2299, 2: 2047}

I have to confess that I "cheated" here: after seeing the problem description for Part 2, I went back and refactored the code for Part 1 in two places:
- `Layout`: Introduced the `crowded` attribute; it had been an inline literal `4`. Also made `deltas` an attribute.
- `next_generation`: Changed `Layout(seats)` to  `type(self)(seats)`.

There was more refactoring and less reuse in Part 2 than I would have liked, but I don't feel like I made bad choices in Part 1.

# Day 12: Rain Risk 

Another problem involving interpreting a kind of assembly language, with navigation instructions for a ship. 

1. Figure out where the navigation instructions lead. What is the Manhattan distance between that location and the ship's starting position?
2. Figure out where the navigation instructions *actually* lead, with the updated interpretation. What is the Manhattan distance between that location and the ship's starting position?

The difference between Part 1 and Part 2 comes down to the initial value of the heading/waypoint, and whether the N/E/W/S commands alter the location or the waypoint. Everything else is the same between the two parts.

In [45]:
in12: List[Instruction] = data(12, lambda line: (line[0], int(line[1:])))

In [46]:
Point = Heading = Tuple[int, int]

directions = dict(N=(0, 1), E=(1, 0), S=(0, -1), W=(-1, 0))

def navigate(instructions, loc=(0, 0), heading=directions['E']) -> Point:
    "Follow instructions to change ship's loc and heading; return final loc."
    for op, n in instructions:
        if   op == 'R': heading = turn(n, *heading)
        elif op == 'L': heading = turn(-n, *heading)
        elif op == 'F': loc = go(n, *loc, *heading)
        else:           loc = go(n, *loc, *directions[op])
    return loc

def turn(degrees, x, y) -> Heading:
    "Turn `degrees` from the current (x, y) heading."
    return (x, y) if degrees % 360 == 0 else turn(degrees - 90, y, -x)
            
def go(n, x, y, dx, dy) -> Point: 
    "Go n steps in the (dx, dy) direction from (x, y)."
    return (x + n * dx, y + n * dy)

def manhatten_distance(point) -> int: return sum(map(abs, point))

def day12_1(instructions): return manhatten_distance(navigate(instructions))

In [47]:
def navigate2(instructions, loc=(0, 0), way=(10, 1)) -> Point:
    "Follow updated instructions to change ship's loc and waypoint; return final loc."
    for op, n in instructions:
        if   op == 'R': way = turn(n, *way)
        elif op == 'L': way = turn(-n, *way)
        elif op == 'F': loc = go(n, *loc, *way)
        else:           way = go(n, *way, *directions[op])
    return loc

def day12_2(instructions): return manhatten_distance(navigate2(instructions))

In [48]:
do(12, 439, 12385)

{1: 439, 2: 12385}

# Day 13: Shuttle Search

A bus with ID *d* leaves the terminal at times 0, *d*, 2*d*, 3*d*, ... You are given bus IDs and an earliest time you can leave.
1. What is the ID of the earliest bus you can take, multiplied by the number of minutes you'll need to wait for that bus?
2. What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?

In [49]:
x = 0
in13: Tuple[int] = (29,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,x,x,x,577,
    x,x,x,x,x,x,x,x,x,x,x,x,13,17,x,x,x,x,19,x,x,x,23,x,x,x,x,x,x,x,601,x,x,x,x,x,x,
    x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,37)

In [50]:
def day13_1(ids, start=1000001):
    "Find the id of the earliest bus after start; return id * wait."
    ids = set(ids) - {x}
    id  = min(ids, key=lambda id: wait(id, start))
    return id * wait(id, start)

def wait(id, t): 
    "How long you have to wait from t for bus id."
    return 0 if t % id == 0 else id - t % id

Here's a brute-force solution for Part 2 that works for the simple test cases:

In [51]:
def day13_2(ids):
    "Find the time where all the buses arrive at the right offsets."
    schedule = {t: id for t, id in enumerate(ids) if id != x}
    step = schedule[0]
    return first(t for t in range(0, maxsize, step) 
                 if all(wait(schedule[i], t) == i for i in schedule))

assert day13_2((7,13,x,x,59,x,31,19)) == 1068781
assert day13_2((1789,37,47,1889)) == 1202161486

However, it is clear this will be too slow for the real input data. Instead of looking at every multiple of the first number, I'll incrementally update the `step` as we go through the numbers. Out of all the puzzles so far, this is the one I had to think most carefully about. For each bus id, we want to find a time where we get that id right, then step the time by a multiple of all the ids encountered so far:

In [52]:
def day13_2(ids):
    "Find the time where all the buses arrive at the right offsets."
    time = 0
    step = 1
    schedule = {t: id for t, id in enumerate(ids) if id != x}
    for t in schedule:
        while wait(schedule[t], time + t):
            time += step
        step *= schedule[t]
    return time

do(13, 174, 780601154795940)

{1: 174, 2: 780601154795940}

# Day 14: Docking Data

Another "interpret assembly code" puzzle, with two different versions of the instructions (which I won't describe here).

1. Execute the initialization program. What is the sum of all values left in memory after it completes?
2. Execute the initialization program using an emulator for a version 2 decoder chip. What is the sum of all values left in memory after it completes?

A *mask* is a bit string but with three possible values at each position, 01X. I could make it into two bitstrings, but I choose to leave it as a `str`.

In [53]:
def parse_docking(line: str) -> tuple:
    "Parse 'mask = XX10' to ('mask', 'XX10') and 'mem[8] = 11' to (8, 11)"
    if line.startswith('mask'):
        return ('mask', line.split()[-1])
    else:
        return ints(line)

in14 = data(14, parse_docking)

In [54]:
Memory = Dict[int, int]

def run_docking(program) -> Memory:
    "Execute the program and return memory."
    mask = bin36(0)
    mem  = defaultdict(int)
    for addr, val in program:
        if addr == 'mask':
            mask = val
        else:
            mem[addr] = int(cat(m if m in '01' else v
                                for m, v in zip(mask, bin36(val))),
                            base=2)
    return mem

def bin36(i) -> str: return f'{i:036b}'

def day14_1(program): return sum(run_docking(program).values())

In [55]:
def day14_2(program): return sum(run_docking2(program).values())
    
def run_docking2(program) -> Memory:
    "Execute the program using version 2 instructions and return memory."
    mask = '0' * 36
    mem  = defaultdict(int)
    for addr, val in program:
        if addr == 'mask':
            mask = val
        else:
            addr = cat(a if m == '0' else '1' if m == '1' else '{}'
                       for m, a in zip(mask, bin36(addr)))
            for bits in product('01', repeat=addr.count('{')):
                mem[int(addr.format(*bits), base=2)] = val
    return mem

In [56]:
do(14, 11884151942312, 2625449018811)

{1: 11884151942312, 2: 2625449018811}

# Day 15: Rambunctious Recitation

This puzzle involves a game where players speak a new number each turn, based on previous numbers.

1. Given your starting numbers, what will be the 2020th number spoken?
2. Given your starting numbers, what will be the 30,000,000th number spoken?

In [57]:
in15 = 10,16,6,0,1,17

In [58]:
def day15_1(starting: Tuple[int], nth=2020) -> int:
    "Return the nth (1-based) number spoken."
    last = starting[-1]
    # `spoken` is a mapping of {number: turn_when_last_spoken}
    spoken = defaultdict(int, {n: t for t, n in enumerate(starting[:-1])})
    for t in range(len(starting), nth):
        new = 0 if last not in spoken else t - 1 - spoken[last]
        spoken[last] = t - 1
        last = new
    return last
    
assert day15_1((0, 3, 6), 2020) == 436

Part 2 involves no changes, but looks for the 30 millionth number. If it had been 3 million, I'd think "no problem!" If it had been 30 billion, I'd think "I need a more efficient solution!" Can 30 million be done in 15 seconds, which  [AoC's about page](https://adventofcode.com/2020/about) says is the maximum run time? I'm not sure; let's try it:

In [59]:
def day15_2(starting): return day15_1(starting, nth=30_000_000)

In [60]:
%time do(15, 412, 243)

CPU times: user 11.5 s, sys: 273 ms, total: 11.8 s
Wall time: 11.8 s


{1: 412, 2: 243}

# Day 16: Ticket Translation

1. Consider the validity of the nearby tickets you scanned. What is the sum of the values that are are not valid for any field?
2. Discard invalid tickets. Use the remaining valid tickets to determine which field is which. Look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

First parse the input file, introducing the class `TicketData` to hold the three parts of the input file (the fields, your ticket, and nearby tickets), and the class `Sets` for a tuple of ranges or other set-like objects, so that we can easily test a number is an element of any one of a number of possibilities. For Part 1, just go through the ticket values and see which values are not in any range. 

In [61]:
TicketData = namedtuple('TicketData', 'fields, your, nearby')

Ticket = ints # A ticket is a tuple of ints

class Sets(tuple):
    "A tuple of set-like objects (such as ranges); supports `in`."
    def __contains__(self, item): return any(item in s for s in self)
    
def parse_ticket_sections(fieldstr: str, your: str, nearby: str) -> TicketData:
    fields = dict(map(parse_ticket_line, fieldstr))
    return TicketData(fields=fields, 
                      your=Ticket(your[1]), 
                      nearby=[Ticket(line) for line in nearby[1:]])

def parse_ticket_line(line: str) -> Tuple[str, Sets]:
    "Parse 'row: 10-20 or 30-40' to ('row', Sets((range(10, 21), range(30, 41))))."
    field = line.split(':')[0]
    a, b, c, d = ints(line.replace('-', ' '))
    return field, Sets((range(a, b + 1), range(c, d + 1)))

in16 = parse_ticket_sections(*data(16, str.splitlines, sep='\n\n'))

In [62]:
def day16_1(ticket_data):
    "The sum of the invalid entries in the nearby tickets."
    ranges = Sets(ticket_data.fields.values())
    return sum(v for ticket in ticket_data.nearby for v in ticket
               if v not in ranges)

For part 2, we're playing a simplified variant of Sudoku:
- First find the valid tickets. 
- Then start with the assumption that any field name is `possible` for any index number in the tickets.
- Determine what field names are invalid for what ticket index numbers.
- Remove the field name from the possibilities for that index, and
   - If there is only one possible field name left, then remove it from all other index positions.

In [63]:
def valid_ticket(ticket, ranges) -> bool: return all(v in ranges for v in ticket)
    
def decode_tickets(ticket_data) -> Dict[str, int]:
    "Return a mapping of {field_name: field_number} (index into ticket)."
    fields, your, nearby = ticket_data
    ranges   = Sets(ticket_data.fields.values())
    valid    = [t for t in nearby + [your] if valid_ticket(t, ranges)]
    possible = [set(fields) for _ in range(len(your))]
    while any(len(p) > 1 for p in possible):
        for field_name, i in invalid_fields(valid, fields):
            possible[i] -= {field_name}
            if len(possible[i]) == 1:
                eliminate_others(possible, i)
    return {field: i for i, [field] in enumerate(possible)}

def invalid_fields(valid, fields) -> Iterable[Tuple[str, int]]:
    "Yield (field_name, field_number) for all invalid fields."
    return ((field_name, i) for ticket in valid for i in range(len(ticket))
            for field_name in fields 
            if ticket[i] not in fields[field_name])

def eliminate_others(possible, i):
    "Eliminate possible[i] from all other possible[j]."
    for j in range(len(possible)):
        if j != i:
            possible[j] -= possible[i]

def day16_2(ticket_data):
    "The product of the 6 fields that start with 'departure'."
    code = decode_tickets(ticket_data)
    return prod(ticket_data.your[code[field]] 
                for field in code if field.startswith('departure'))

In [64]:
do(16)

{1: 21071, 2: 3429967441937}

# Day 17: Conway Cubes

Now we are explicitly playing *Life*, but in three dimensions not two. I've coded this before; I'll adapt my [old version](Life.ipynb) to three dimensions. My implementation represents a generation as the set of active cell coordinates.

In [65]:
in17: Picture = '''
##.#....
...#...#
.#.#.##.
..#.#...
.###....
.##.#...
#.##..##
#.####..'''.strip().splitlines()

In [66]:
Cell = tuple # of (x, y, z) int coordinates

def day17_1(picture, n=6):
    "How many cells are active in the nth generation?"
    cells = parse_cells(picture)
    for g in range(n):
        cells = next_generation(cells)
    return len(cells)

def parse_cells(picture, active='#') -> Set[Cell]:
    "Convert a 2-d picture into a set of 3-d active cells."
    return {(x, y, 0) for (y, row) in enumerate(picture)
            for x, cell in enumerate(row) if cell is active}

def next_generation(cells) -> Set[Cell]:
    """The set of live cells in the next generation."""
    return {cell for cell, count in neighbor_counts(cells).items()
            if count == 3 or (count == 2 and cell in cells)}

def neighbor_counts(cells) -> Dict[Cell, int]:
    "The number of live neighbors for each cell that has neighbors."
    return Counter(flatten(map(neighbors, cells)))

deltas3d = set(product((-1, 0, +1), repeat=3)) - {(0, 0, 0)}

def neighbors(cell) -> List[cell]:
    "All adjacent neighbors of cell in three dimensions."
    return [tuple(map(operator.add, cell, delta)) 
            for delta in deltas3d]

Part 2 asks us to move to 4 dimensions. I'll generalize the previous code to work in 3 or 4 dimensions:

In [67]:
def parse_cells(picture, d=3, active='#') -> Set[Cell]:
    "Convert a 2-d picture into a set of d-dimensional active cells."
    return {(x, y, *(d - 2) * (0,) )
            for (y, row) in enumerate(picture)
            for x, cell in enumerate(row) if cell is active}

def day17_1(picture): return day17_2(picture, n=6, d=3)

def day17_2(picture, n=6, d=4):
    "How many cells are active in the nth generation in a d-dimensional world?"
    cells = parse_cells(picture, d=d)
    for g in range(n):
        cells = next_generation(cells)
    return len(cells)

deltas = [set(product((-1, 0, +1), repeat=d)) - {(0,) * d}
          for d in range(5)]

def neighbors(cell) -> List[cell]:
    "All adjacent neighbors of cell in all dimensions."
    return [tuple(map(operator.add, cell, delta)) 
            for delta in deltas[len(cell)]]

In [68]:
do(17, 291, 1524)

{1: 291, 2: 1524}

# Day 18: Operation Order

At first I thought I could just apply `eval` to each line, but alas, the operation order is different. I could have used a parsing framework, but I decided to do it all from scratch.

1. All operations are done left-to-right. Evaluate the expression on each line of the homework; what is the sum of the resulting values?
2. Addition is done before multiplication. What do you get if you add up the results of evaluating the homework problems using these new rules?

In [69]:
def parse_expr(line) -> tuple: 
    "Parse an expression: '2 + 3 * 4' => (2, '+', 3, '*', 4)."
    return ast.literal_eval(re.sub('([+*])', r",'\1',", line))

in18 = data(18, parse_expr)

In [70]:
operators = {'+': operator.add, '*': operator.mul}

def evaluate(expr) -> int:
    "Evaluate an expression under left-to-right rules."
    if isinstance(expr, int):
        return expr
    else:
        a, op, b, *rest = expr
        x = operators[op](evaluate(a), evaluate(b))
        return x if not rest else evaluate((x, *rest))
    
def day18_1(exprs): return sum(map(evaluate, exprs))

In [71]:
def evaluate2(expr) -> int:
    "Evaluate an expression under addition-first rules."
    if isinstance(expr, int):
        return expr
    else:
        expr = list(expr)  # Make expr mutable
        while '+' in expr: # Change [... 3, '+', 4 ...] to [... 7 ...]
            i = expr.index('+')
            expr[i-1:i+2] = [evaluate2(expr[i - 1]) + evaluate2(expr[i + 1])]
        return prod(evaluate2(x) for x in expr if x not in operators)
    
def day18_2(exprs): return sum(map(evaluate2, exprs))

In [72]:
do(18, 3885386961962, 112899558798666)

{1: 3885386961962, 2: 112899558798666}

# Summary, and Timing

So far, two of the puzzles run slowly (but under the 15 second cap). All the other 16 puzzles combined run in under a second total.

In [73]:
advent = set(range(1, 26))
slow   = {11, 15}

def doall(days): return {i: do(i) for i in days}

%time doall(advent - slow)   

CPU times: user 817 ms, sys: 5.42 ms, total: 823 ms
Wall time: 825 ms


{1: {1: 787776, 2: 262738554},
 2: {1: 383, 2: 272},
 3: {1: 167, 2: 736527114},
 4: {1: 237, 2: 172},
 5: {1: 906, 2: 519},
 6: {1: 6530, 2: 3323},
 7: {1: 103, 2: 1469},
 8: {1: 1521, 2: 1016},
 9: {1: 776203571, 2: 104800569},
 10: {1: 2346, 2: 6044831973376},
 12: {1: 439, 2: 12385},
 13: {1: 174, 2: 780601154795940},
 14: {1: 11884151942312, 2: 2625449018811},
 16: {1: 21071, 2: 3429967441937},
 17: {1: 291, 2: 1524},
 18: {1: 3885386961962, 2: 112899558798666},
 19: {},
 20: {},
 21: {},
 22: {},
 23: {},
 24: {},
 25: {}}

In [74]:
%time do(11)

CPU times: user 10.6 s, sys: 41.2 ms, total: 10.6 s
Wall time: 10.7 s


{1: 2299, 2: 2047}

In [75]:
%time do(15)

CPU times: user 10.9 s, sys: 254 ms, total: 11.1 s
Wall time: 11.2 s


{1: 412, 2: 243}